In [1]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/sjsimmo2/DataMining-Fall/refs/heads/master/orderData.csv')

In [3]:
# seperate into different types of item in order.
meats = [
    "Filet Mignon",
    "Sea Bass",
    "Pork Tenderloin",
    "Pork Chop",
    "Salmon",
    "Duck Breast",
    "Swordfish",
    "Roast Chicken"
]

sides = [
    "Seasonal Veg",
    "Bean Trio",
    "Roasted Root Veg",
    "Warm Goat Salad",
    "Roasted Potatoes",
    "Caesar Salad",
    "Mashed Potatoes"
]

white_wine =  [
    "Brancott Pinot Grigio",
    "Cantina Pinot Bianco",
    "Duckhorn Chardonnay",
    "Helben Blanc",
    "Innocent Bystander Sauvignon Blanc",
    "Oyster Bay Sauvignon Blanc",
    "Total Recall Chardonnay"
]

red_wine = [
    "Adelsheim Pinot Noir",
    "Blackstone Merlot",
    "Echeverria Gran Syrah",
    "Louis Rouge",
    "Single Vineyard Malbec",
    "Three Rivers Red"
]


wines = [
    "Duckhorn Chardonnay",
    "Adelsheim Pinot Noir",
    "Blackstone Merlot",
    "Total Recall Chardonnay",
    "Single Vineyard Malbec",
    "Innocent Bystander Sauvignon Blanc",
    "Oyster Bay Sauvignon Blanc",
    "Echeverria Gran Syrah",
    "Brancott Pinot Grigio",
    "Cantina Pinot Bianco",
    "Louis Rouge",
    "Helben Blanc",
    "Three Rivers Red"
]

In [4]:
#create a df with only the meat and wine pairings
df_meat_wine = df[df['item'].isin(meats + wines)]

#create a dummy varriable for each item
df_1 = pd.get_dummies(df_meat_wine["item"])*1

#add the original order number to the new df
df_1["orderNo"] = df_meat_wine["orderNo"]
#add the original seat number to the new df
df_1['seatNo'] = df_meat_wine['seatNo']
#df_1["order_seatNo"] = df['orderNo'].astype(str) + "_" + df['seatNo'].astype(str)

#group by orderNo and seatNo, then calculates the maximum value for each col
df_1 = df_1.groupby(['orderNo', 'seatNo']).max() # why do .max? see below

#convert the dummy vars back to boolean 
preprocessed_df = df_1.map(bool)

preprocessed_df.head()


# Dummy Variables: You’ve created dummy variables for each item in the meats and wines lists, where each column corresponds to an item and contains a 1 if that item was ordered and a 0 otherwise.

# Grouping: By grouping the data by orderNo and seatNo, you are aggregating the orders at the level of each unique combination of orderNo and seatNo. This step ensures you combine all the items ordered by the same person (seat) during the same order.

# Maximum Calculation: When you apply .max(), you are essentially finding out if a particular item was ordered within that group. If any instance of the item was ordered, the dummy variable for that item will contain a 1; otherwise, it will remain 0. Taking the maximum ensures that for each item (meat or wine), a 1 appears if it was ordered, and 0 if not.

# Final Step (Boolean Conversion): After this, converting the dummy variables back to boolean (True for 1, False for 0) makes the dataframe easier to interpret in terms of whether a given item was ordered at a specific seat for an order.

# In summary, you're finding the maximum value to capture whether a particular meat or wine item was ordered at least once in the context of a specific orderNo and seatNo combination.

Adelsheim Pinot Noir  Blackstone Merlot  \
orderNo seatNo                                            
122314  1                      False              False   
        2                      False              False   
        3                      False              False   
        4                      False              False   
        5                      False              False   

                Brancott Pinot Grigio  Cantina Pinot Bianco  Duck Breast  \
orderNo seatNo                                                             
122314  1                       False                 False        False   
        2                       False                 False        False   
        3                       False                 False        False   
        4                       False                 False        False   
        5                       False                 False         True   

                Duckhorn Chardonnay  Echeverria Gran Syrah  Filet Mignon  \
orderNo seatNo                                                             
122314  1                     False                  False         False   
        2                     False                  False         False   
        3                     False                  False         False   
        4                     False                  False         False   
        5                     False                  False         False   

                Helben Blanc  Innocent Bystander Sauvignon Blanc  ...  \
orderNo seatNo                                                    ...   
122314  1              False                               False  ...   
        2              False                               False  ...   
        3              False                               False  ...   
        4              False                               False  ...   
        5              False                                True  ...   

                Oyster Bay Sauvignon Blanc  Pork Chop  Pork Tenderloin  \
orderNo seatNo                                                           
122314  1                             True      False            False   
        2                            False       True            False   
        3                             True      False            False   
        4                            False      False            False   
        5                            False      False            False   

                Roast Chicken  Salmon  Sea Bass  Single Vineyard Malbec  \
orderNo seatNo                                                            
122314  1               False    True     False                   False   
        2               False   False     False                   False   
        3               False   False      True                   False   
        4               False   False      True                   False   
        5               False   False     False                   False   

                Swordfish  Three Rivers Red  Total Recall Chardonnay  
orderNo seatNo                                                        
122314  1           False             False                    False  
        2           False              True                    False  
        3           False             False                    False  
        4           False             False                     True  
        5           False             False                    False  

[5 rows x 21 columns]

In [5]:
#apriori algorithm
food_wine_assoc = apriori(preprocessed_df, min_support = 0.001, use_colnames = True)

#association rules
out_rules = association_rules(food_wine_assoc,metric = "confidence", min_threshold = 0.1)

#select columns from the output
out_rules2=out_rules[['antecedents','consequents','support','confidence','lift']]

out_rules2

,antecedents,consequents,support,confidence,lift
0,(Adelsheim Pinot Noir),(Duck Breast),0.016476,0.121847,1.173565
1,(Duck Breast),(Adelsheim Pinot Noir),0.016476,0.158686,1.173565
2,(Adelsheim Pinot Noir),(Filet Mignon),0.049650,0.367191,2.087867
3,(Filet Mignon),(Adelsheim Pinot Noir),0.049650,0.282315,2.087867
4,(Adelsheim Pinot Noir),(Pork Chop),0.013826,0.102251,0.710175
...,...,...,...,...,...
80,(Three Rivers Red),(Sea Bass),0.004854,0.260931,1.616936
81,(Total Recall Chardonnay),(Sea Bass),0.015990,0.152147,0.942823
82,(Three Rivers Red),(Swordfish),0.001928,0.103667,1.062355
83,(Total Recall Chardonnay),(Swordfish),0.017447,0.166001,1.701136


In [6]:
# only filter the main entree. Want "once they order main entree, let's see what kind of wine they get."
meats_antecedents = out_rules2[out_rules2['antecedents'].apply(lambda x: list(x)[0] in meats)]
meats_antecedents

,antecedents,consequents,support,confidence,lift
1,(Duck Breast),(Adelsheim Pinot Noir),0.016476,0.158686,1.173565
3,(Filet Mignon),(Adelsheim Pinot Noir),0.049650,0.282315,2.087867
6,(Pork Tenderloin),(Adelsheim Pinot Noir),0.043734,0.299336,2.213742
8,(Duck Breast),(Blackstone Merlot),0.032886,0.316740,2.845736
10,(Filet Mignon),(Blackstone Merlot),0.062729,0.356679,3.204565
12,(Roast Chicken),(Brancott Pinot Grigio),0.008448,0.173118,2.947147
15,(Sea Bass),(Brancott Pinot Grigio),0.016856,0.104455,1.778223
17,(Swordfish),(Brancott Pinot Grigio),0.014705,0.150692,2.565370
21,(Salmon),(Cantina Pinot Bianco),0.013328,0.108826,2.308330
30,(Duck Breast),(Single Vineyard Malbec),0.010533,0.101453,0.992692


In [7]:
# We can also separate wines into red wines:
red_wine_consequent = out_rules2[out_rules2['consequents'].apply(lambda x: list(x)[0] in red_wine)]
red_wine_consequent

# 4 types of fattier dish: duck, filet mignon, pork(2)

,antecedents,consequents,support,confidence,lift
1,(Duck Breast),(Adelsheim Pinot Noir),0.016476,0.158686,1.173565
3,(Filet Mignon),(Adelsheim Pinot Noir),0.049650,0.282315,2.087867
6,(Pork Tenderloin),(Adelsheim Pinot Noir),0.043734,0.299336,2.213742
8,(Duck Breast),(Blackstone Merlot),0.032886,0.316740,2.845736
10,(Filet Mignon),(Blackstone Merlot),0.062729,0.356679,3.204565
30,(Duck Breast),(Single Vineyard Malbec),0.010533,0.101453,0.992692
43,(Filet Mignon),(Echeverria Gran Syrah),0.024648,0.140151,2.322632
46,(Filet Mignon),(Single Vineyard Malbec),0.037228,0.211680,2.071241
57,(Pork Chop),(Louis Rouge),0.016555,0.114978,3.397336
69,(Pork Chop),(Single Vineyard Malbec),0.024399,0.169461,1.658130


In [8]:
# We can also separate wines into white wines:
white_wine_consequent = out_rules2[out_rules2['consequents'].apply(lambda x: list(x)[0] in white_wine)]
white_wine_consequent

# chicken, fish, pork (people order both red and white wine for pork)

,antecedents,consequents,support,confidence,lift
12,(Roast Chicken),(Brancott Pinot Grigio),0.008448,0.173118,2.947147
15,(Sea Bass),(Brancott Pinot Grigio),0.016856,0.104455,1.778223
17,(Swordfish),(Brancott Pinot Grigio),0.014705,0.150692,2.565370
21,(Salmon),(Cantina Pinot Bianco),0.013328,0.108826,2.308330
32,(Pork Chop),(Duckhorn Chardonnay),0.033004,0.229227,1.490633
34,(Pork Tenderloin),(Duckhorn Chardonnay),0.035510,0.243042,1.580466
35,(Roast Chicken),(Duckhorn Chardonnay),0.005982,0.122581,0.797124
36,(Salmon),(Duckhorn Chardonnay),0.018207,0.148672,0.966792
39,(Sea Bass),(Duckhorn Chardonnay),0.040232,0.249309,1.621221
41,(Swordfish),(Duckhorn Chardonnay),0.017538,0.179728,1.168749
